## House Price Prediction
### Michał Lange
####  Data exploration and machine learning in scientific research


The following python notebook will train an ML model based to predict House prices based off of data from the following kaggle competition: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques. This document will delineate each step of the pipeline as well as showing the associated python code and results.

1.) The first step is to import all necessary python packages.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

Now let's import our data and take a look at the column values to see what we are working with.

In [2]:
train_path = "./data/train.csv"
test_path = "./data/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Display the parameters of the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In total we have 79 total features we can use for prediction, however we should drop "Id" from both datasets as it is just a building indicator and has no predictive capabilities.

In [3]:
train = train.drop('Id', axis=1)
test = test.drop('Id', axis=1)

## Exploratory Data Analysis

Let's check for missing values in our dataset to understand what data cleaning will be required.

In [4]:
# Check for missing values in training data
missing_train = train.isnull().sum()
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)

# Calculate percentage of missing values
missing_percent = (missing_train / len(train)) * 100

print(f"Number of features with missing values: {len(missing_train)}")

# Create bar chart visualization
fig = px.bar(
    x=missing_percent.index,
    y=missing_percent.values,
    labels={'x': 'Features', 'y': 'Percentage Missing (%)'},
    title='Missing Values by Feature'
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=500,
    showlegend=False
)

fig.show()

Number of features with missing values: 19


There are several things here of note but let's address them one by one. In the case of MasVnrArea and Electrical the missing percentage is low enough that we can impute the missing values with the median for numerical and mode for categorical.

In [7]:
# Impute MasVnrArea and Electrical with median/mode
train['MasVnrArea'] = train['MasVnrArea'].fillna(train['MasVnrArea'].median())
train['Electrical'] = train['Electrical'].fillna(train['Electrical'].mode()[0])

# Do the same for test set
test['MasVnrArea'] = test['MasVnrArea'].fillna(test['MasVnrArea'].median())
test['Electrical'] = test['Electrical'].fillna(test['Electrical'].mode()[0])

Now we can see we have very similar groups of features that start with "Bsmt" and "Garage". This most likely indicates a group of houses that have missing basements or garages. To test this theory we can first check if those features are missing in the same rows.

In [8]:
# Get all basement and garage features
bsmt_features = [col for col in train.columns if col.startswith('Bsmt')]
garage_features = [col for col in train.columns if col.startswith('Garage')]

print(f"Basement features: {bsmt_features}")
print(f"Garage features: {garage_features}")

# Check if missing values occur in the same rows for basement features
bsmt_missing_together = train[bsmt_features].isnull().sum(axis=1)
print(f"\nHouses with ALL basement features missing: {(bsmt_missing_together == len(bsmt_features)).sum()}")
print(f"Houses with SOME basement features missing: {((bsmt_missing_together > 0) & (bsmt_missing_together < len(bsmt_features))).sum()}")

# Check if missing values occur in the same rows for garage features
garage_missing_together = train[garage_features].isnull().sum(axis=1)
print(f"\nHouses with ALL garage features missing: {(garage_missing_together == len(garage_features)).sum()}")
print(f"Houses with SOME garage features missing: {((garage_missing_together > 0) & (garage_missing_together < len(garage_features))).sum()}")

Basement features: ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtFullBath', 'BsmtHalfBath']
Garage features: ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']

Houses with ALL basement features missing: 0
Houses with SOME basement features missing: 39

Houses with ALL garage features missing: 0
Houses with SOME garage features missing: 81


We can see there are some houses with partial basement data missing. Let's visualize which specific basement features are missing across these houses using a heatmap.

In [11]:
# Get houses with partial basement data missing
partial_bsmt_missing = (bsmt_missing_together > 0) & (bsmt_missing_together < len(bsmt_features))
partial_bsmt_houses = train[partial_bsmt_missing]

# Create a heatmap showing which basement features are missing
missing_matrix = partial_bsmt_houses[bsmt_features].isnull().astype(int)

# Reset index to show house numbers instead of IDs
missing_matrix_reset = missing_matrix.reset_index(drop=True)

fig = px.imshow(
    missing_matrix_reset.T,
    labels=dict(x="House Number", y="Basement Features", color="Missing"),
    y=bsmt_features,
    color_continuous_scale=['lightgreen', 'darkred'],
    title='Missing Basement Features Heatmap (Partial Missing Only)'
)

fig.update_layout(
    height=400,
    xaxis_title="Houses with Partial Basement Data",
    yaxis_title="Basement Features"
)

fig.show()


Since some houses have only partial basement data missing rather than all basement features missing together, this suggests the missingness is not simply due to absence of a basement. The missing values may reflect incomplete data collection during the appraisal process. Rather than imputing with mode, which could introduce bias, we'll create a new "Missing" category to preserve this information.

In [ ]:
# Fill missing basement features with "Missing" category
for feature in bsmt_features:
    train[feature] = train[feature].fillna('Missing')
    test[feature] = test[feature].fillna('Missing')


Let's take a look now at the Garage features to see if we have the same issue.